In [1]:
!git clone https://github.com/IlyaSk10/POS_Tagger_with_using_CNN/
%cd POS_Tagger_with_using_CNN
!unzip SYNTAGRUS_texts.zip
!head syntagrus_full.ud
from data_preparation.converter_from_ud_to_txt import UDConverter
UDConverter.convert_from_conllu("syntagrus_full.ud", "syntagrus_fixed.txt")
!head syntagrus_fixed.txt
text_data= "syntagrus_fixed.txt"

Cloning into 'POS_Tagger_with_using_CNN'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 26 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (26/26), done.
/content/POS_Tagger_with_using_CNN
Archive:  SYNTAGRUS_texts.zip
  inflating: syntagrus_full.ud       
1	Начальник	начальник	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
2	областного	областной	ADJ	Case=Gen|Degree=Pos|Gender=Neut|Number=Sing
3	управления	управление	NOUN	Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing
4	связи	связь	NOUN	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
5	Семен	семен	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
6	Еремеевич	еремеевич	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
7	был	быть	VERB	Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act
8	человек	человек	NOUN	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing
9	простой	простой	ADJ	Case=Nom|Degree=Pos|Gen

In [0]:
def load_from_file(file):
  with open(file,'r',encoding='utf-8') as f:
    c=f.read().split('\n')
  return c

In [0]:
dataset=load_from_file(text_data)


In [5]:
dataset=[line for line in dataset if line!='']
MAX_LEN_TOKEN=max(len(line.split('\t')[0]) for line in dataset)
NUMBER_UNIQUE_TOKEN=len(set(line.split('\t')[0] for line in dataset if line.split('\t')[2]!='PUNCT'))
print('Максимальная длина токена', MAX_LEN_TOKEN)
print('Количество уникальных токенов' , NUMBER_UNIQUE_TOKEN)

Максимальная длина токена 31
Количество уникальных токенов 112875


In [6]:
s = ['<NOTAG>'] + sorted({token.split('\t')[2] for token in dataset})
label2id = {label: i for i, label in enumerate(s)}
print('Метки частей речи' , label2id)

Метки частей речи {'<NOTAG>': 0, 'ADJ': 1, 'ADP': 2, 'ADV': 3, 'CONJ': 4, 'DET': 5, 'H': 6, 'INTJ': 7, 'NOUN': 8, 'NUM': 9, 'PART': 10, 'PRON': 11, 'PUNCT': 12, 'VERB': 13}


In [0]:
from collections import Counter
def build_vocab(text,pad_symbol='<PAD>',most_freq_symbols=None):
  sentence=' '.join(line.split('\t')[0] for line in text)
  mydict = dict((j, i) for i, j in enumerate(set(sentence.lower())))
  mydict.update({pad_symbol:max(mydict.values())+1})

  def counter(sentence,most_freq_symbols):
    if most_freq_symbols==None:
       most_freq_symbols=5
    return Counter(list(sentence)).most_common(most_freq_symbols)    

  return 'Symbols in vocabulary',mydict.items(),  'Most frequently symbols', counter(sentence,most_freq_symbols)

In [13]:
build_vocab(dataset[:10],most_freq_symbols=10)

('Symbols in vocabulary',
 dict_items([('у', 0), (',', 1), ('б', 2), ('с', 3), ('м', 4), ('й', 5), ('ь', 6), ('з', 7), ('т', 8), ('г', 9), ('н', 10), (' ', 11), ('я', 12), ('ы', 13), ('к', 14), ('о', 15), ('ч', 16), ('р', 17), ('в', 18), ('е', 19), ('и', 20), ('л', 21), ('а', 22), ('п', 23), ('<PAD>', 24)]),
 'Most frequently symbols',
 [(' ', 9),
  ('е', 8),
  ('о', 6),
  ('л', 5),
  ('а', 4),
  ('н', 4),
  ('и', 4),
  ('в', 4),
  ('ч', 3),
  ('с', 3)])